In [2]:
!pip install netket

In [4]:
import netket as nk
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

def get_lowest_data(n_sites_list):

    results = []

    for n in n_sites_list:
        print(f"--- Analyzing lowest e_val for N = {n} ---")

        # Lattice and Hamiltonian

        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, total_sz=0, N=graph.n_nodes)
        hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph)
        e_exact = None
        ed_result = nk.exact.lanczos_ed(hamiltonian, k=1)
        e_exact = float(ed_result[0])

        #print(f"Lanczos (exact): {e_exact}")
        #Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=float) #RBM DENSITY COEFFICIENT
        sampler = nk.sampler.MetropolisExchange(hi, graph=graph) #using Metropolis Exchange bc total_sz = 0

        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)


        # Neel state

        n_chains = vstate.sampler.n_chains
        neel = np.ones(n)
        neel[1::2] = -1
        neel_samples = np.tile(neel, (n_chains, 1)).astype(np.int8)
        vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(neel_samples))

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.01)  #LEARNING RATE
        #sr = nk.optimizer.SR(diag_shift=0.1)
        #vmc = nk.driver.VMC(hamiltonian, optimizer, variational_state=vstate, preconditioner=sr)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=200)
        e_vmc = float(vstate.expect(hamiltonian).mean.real)
        results.append({
            "N": n,
            "VMC": e_vmc,
            "Lanczos": e_exact,
            "diff": abs(e_vmc - e_exact),
        })

    return results

def get_highest_data(n_sites_list):

    results = []

    for n in n_sites_list:
        print(f"--- Analyzing Highest e_val for N = {n} ---")

        # Lattice and Hamiltonian

        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)
        hamiltonian = -1 * nk.operator.Heisenberg(hilbert=hi, graph=graph)

        e_exact = None
        ed_result = nk.exact.lanczos_ed(hamiltonian, k=1)
        e_exact = -1 * float(ed_result[0])

        #Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=float) #RBM DENSITY COEFFICIENT
        sampler = nk.sampler.MetropolisLocal(hi)  #using Metropolis Local sampler instead of exchange bc total_sz != 0
        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)


      #all 1 for initial chains option, NOT neel state for highest e_val
        n_chains = vstate.sampler.n_chains
        ferro = np.ones(n)
        ferro_samples = np.tile(ferro, (n_chains, 1)).astype(np.int8)
        #vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(ferro_samples)) #tester

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.01) #LEARNING RATE
        sr = nk.optimizer.SR(diag_shift=0.1)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=500)
        e_vmc = -1*float(vstate.expect(hamiltonian).mean.real)
        results.append({
            "N": n,
            "VMC": e_vmc,
            "Lanczos": e_exact,
            "diff": abs(e_vmc - e_exact),

        })

    return results

def printer(data):
  for d in data:
    status = f"Diff: {d['diff']:.8f}" if d['diff'] is not None else "Exact not computed"
    print(f"N={d['N']} | VMC: {d['VMC']:.8f} | Lanczos: {d['Lanczos']:.8f} | Abs Difference: {d['diff']:.8f} \n")

n_sizes = [6,8,10]

for n in n_sizes:
  data = get_lowest_data([n])
  printer(data)
  data = get_highest_data([n])
  printer(data)








--- Analyzing lowest e_val for N = 6 ---
Automatic SR implementation choice:  QGT


  0%|          | 0/200 [00:00<?, ?it/s]

N=6 | VMC: -11.21120015 | Lanczos: -11.21110255 | Abs Difference: 0.00009760 

--- Analyzing Highest e_val for N = 6 ---
Automatic SR implementation choice:  QGT


  0%|          | 0/500 [00:00<?, ?it/s]

N=6 | VMC: 6.00000000 | Lanczos: 6.00000000 | Abs Difference: 0.00000000 

--- Analyzing lowest e_val for N = 8 ---
Automatic SR implementation choice:  QGT


...

N=8 | VMC: 8.00000000 | Lanczos: 8.00000000 | Abs Difference: 0.00000000 

--- Analyzing lowest e_val for N = 10 ---
Automatic SR implementation choice:  QGT


  0%|          | 0/200 [00:00<?, ?it/s]

N=10 | VMC: -18.06308777 | Lanczos: -18.06178542 | Abs Difference: 0.00130236 

--- Analyzing Highest e_val for N = 10 ---
Automatic SR implementation choice:  QGT


  0%|          | 0/500 [00:00<?, ?it/s]

N=10 | VMC: 10.00000000 | Lanczos: 10.00000000 | Abs Difference: 0.00000000 

